Movie polarity classification

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import sys
sys.path.append('/home/tim/Desktop/Data_science/fastai/')

from fastai.learner import *

import torchtext
from torchtext import vocab, data
from torchtext.datasets import language_modeling

from fastai.rnn_reg import *
from fastai.rnn_train import *
from fastai.nlp import *
from fastai.lm_rnn import *

import dill as pickle
import spacy

/home/tim/anaconda/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/tim/anaconda/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/tim/anaconda/envs/fastai/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
/home/tim/anaconda/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/tim/anaconda/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed

In [46]:
PATH = '/home/tim/Desktop/Data_science/fastai/projects/part_1_material/RNNs/data/movie_review_2class/'
trn_path = 'trn/all'
val_path = 'val/all'

In [47]:
train_pos_files = !ls {PATH}{trn_path}/pos
trin_neg_files = !ls {trn_path}/neg
val_pos_files = !ls {val_path}/pos
val_neg_files = !ls {val_path}/neg

Ok let's look at a positive movie review

In [48]:
' '.join([part for part in test_review])

'films adapted from comic books have had plenty of success , whether they\'re about superheroes ( batman , superman , spawn ) , or geared toward kids ( casper ) or the arthouse crowd ( ghost world ) , but there\'s never really been a comic book like from hell before .  for starters , it was created by alan moore ( and eddie campbell ) , who brought the medium to a whole new level in the mid \'80s with a 12-part series called the watchmen .  to say moore and campbell thoroughly researched the subject of jack the ripper would be like saying michael jackson is starting to look a little odd .  the book ( or " graphic novel , " if you will ) is over 500 pages long and includes nearly 30 more that consist of nothing but footnotes .  in other words , don\'t dismiss this film because of its source .  if you can get past the whole comic book thing , you might find another stumbling block in from hell\'s directors , albert and allen hughes .  getting the hughes brothers to direct this seems almo

In [49]:
tokenizer = spacy.load('en')

In [50]:
TEXT = data.Field(lower=True, tokenize="spacy")

In [51]:
bs = 64
bptt = 70

In [52]:
FILES = dict(train=trn_path, validation=val_path, test=val_path)
md = LanguageModelData.from_text_files(PATH, TEXT, **FILES, bs=bs, bptt=bptt, min_freq=10)

Now we can see that the `TEXT` object that we had instansiated above has the properties of 
`TEXT.vocab.itos` (int-to-string) which is the vocabulary for the corpus (i.e. all the words) as well as the mapping dictionary
from word to token via `TEXT.vocab.stoi` (string-to-int). We can see that these provide methods for mapping between tokens and strings, as follows:

In [82]:
print(TEXT.vocab.itos[:12])
print(TEXT.vocab.stoi['the'])

['<unk>', '<pad>', ',', 'the', '.', 'a', 'and', 'of', 'to', 'is', 'in', "'s"]
3


What we can do now is to dump the tokenized text data to a `.pickle` file so that we can read this later instead of having to do the tokenization again.


In [61]:
#pickle.dump(TEXT, open(f'{PATH}models/TEXT.pkl', 'wb'))

Now we can look at some information about the movie reviews such as the # batches in the data set, # of tokens in the vocab, # of tokens in the training set, and the # of sentences.

In [74]:
print('Number of batches: {}'.format(len(md.trn_dl)))
print('Number of tokens in vocab: {}'.format(md.nt))
print('Number of tokens in training set: {}'.format(len(md.trn_ds)))
print('Number of sentences: {}'.format(len(md.trn_ds[0].text)))

Number of batches: 277
Number of tokens in vocab: 8162
Number of tokens in training set: 1
Number of sentences: 1245853


Some things to note about the LanguageModelData object: 1) There is only one item in each dataset:

In [96]:
md.trn_ds[0].text[:12]

['movie',
 'concepts',
 'are',
 'often',
 'pitched',
 'to',
 'producers',
 'with',
 'mathematical',
 'formulas',
 'involving',
 'successful']

2) The LanguageModelData object will create batches with 64 columns (the batch size), and varying sequence lengths around 70 (the bptt we set above). Furthermore, the batches will contain the exact same number of data as labels, but offset by one, because we are trying to predict the next word in the sequence. The labels are a 1D flattened array:

In [98]:
next(iter(md.trn_dl))

(Variable containing:
    34    15     9  ...     45    46     2
  4677     3     5  ...     24     0     6
    28    15  1971  ...    182  2495  4374
        ...          ⋱          ...       
    17    14     4  ...     77     5     2
   378  5180  2039  ...   2015   272    24
  3229     3     2  ...   1177     7     4
 [torch.LongTensor of size 78x64], Variable containing:
  4677
     3
     5
   ⋮  
     4
  2525
   965
 [torch.LongTensor of size 4992])

Let's set the other parameters needed for fast.ai LanguageModels, mainly the embedding dimensions, the number of hidden units per layer,  and the number of hidden layers.

In [105]:
# Embedding size
emb_sz = 200
# Number of hidden units
nh = 250
# Number of hidden layers
nl = 3

Next we want to define the optimizer we want to use, as a default use Adam with the momentum set to less than the default of 0.9.

In [106]:
opt = partial(optim.Adam, betas=(0.7, 0.99))

In [109]:
learner = md.get_model(opt_fn=opt, emb_sz=emb_sz, n_hid=nh,
                       n_layers=nl, dropouti=0.05, dropout=0.05,
                       wdrop=0.1, dropoute=0.02, dropouth=0.05)

We can also set a regularization function for this type of learner as well as a clipping percentage for gradient clipping

In [110]:
learner.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
learner.clip = 0.3

In [ ]:
learner.fit(lrs=1e-3, n_cycle=1, wds=1e-6)

  1%|          | 3/277 [00:25<40:48,  8.94s/it, loss=8.99]